In [1]:
import os
import sys

sys.path.append("../../common/src")
sys.path.append("../src")
os.chdir("../src")

In [2]:
from langchain.agents import Tool, AgentExecutor, BaseMultiActionAgent, AgentOutputParser
from langchain.tools.gmail.utils import build_resource_service, get_gmail_credentials
from langchain.agents.agent_toolkits import GmailToolkit
from langchain.agents import initialize_agent, AgentType
from langchain.agents.chat.base import ChatAgent
from langchain.prompts import StringPromptTemplate, ChatPromptTemplate, PromptTemplate
from langchain.llms import OpenAI, VertexAI
from langchain.chat_models import ChatVertexAI, ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.chains import LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re

In [3]:
!export PROJECT_ID="gcp-mira-demo"
project = "gcp-mira-demo"
os.environ["PROJECT_ID"] = project

In [4]:
import config
from config import (VERTEX_LLM_TYPE_BISON_CHAT,
                    VERTEX_LLM_TYPE_BISON_TEXT,
                    OPENAI_LLM_TYPE_GPT3_5,
                    OPENAI_LLM_TYPE_GPT4,
                    COHERE_LLM_TYPE,
                    LLM_BACKEND_ROBOT_USERNAME,
                    LLM_BACKEND_ROBOT_PASSWORD,
                    OPENAI_API_KEY)

In [5]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [6]:
#  email_template = """
 #   You are worker working for NYS State Medicaid Agency. Write a email to recipient: {recipient} \n\n 
  #  \n\n Use text delimited by triple backticks to create the email body text:'''{email_body}'''
    
   # """

In [7]:
 email_template = """
    You are worker working for {state} State Medicaid Agency. Create only the email message body for recipient: {recipient} \n\n 
    \n\n Use text delimited by triple backticks to create the email body text:'''{email_body}'''
    
    """

In [8]:
email_body_prompt = ChatPromptTemplate.from_template(email_template)

In [9]:
#email_body_prompt.messages[0].prompt.input_variables

In [10]:
llm_chain = LLMChain(prompt=email_body_prompt, llm=llm)

In [11]:
prompt_string = """Create an email to this medicaid applicant that is missing income verification asking them to email a pay stub from their employers"""

In [12]:
sender='mira-admin@lavjain.altostrat.com'

In [13]:
recipient='sumeetvij@gmail.com'

In [14]:
state='NYS'

In [15]:
message = llm_chain.run(sender=sender, state=state,recipient=recipient,email_body=prompt_string)
print(f"{message}")


Dear Sumeet Vij, 

We are writing to you in regards to your application for New York State Medicaid. We have noticed that we are missing income verification from you. 

In order to complete your application, we kindly ask that you email a pay stub from your employer to us. 

Thank you for your cooperation. 

Sincerely, 
NYS State Medicaid Agency


In [16]:
 subject_template = """
    Create a subject for the following email: {email_body} \n\n 
    
    """

In [17]:
subject_prompt = ChatPromptTemplate.from_template(subject_template)

In [18]:
llm_chain = LLMChain(prompt=subject_prompt, llm=llm)

In [19]:
subject = llm_chain.run(email_body=message)
print(f"{subject}")

 Subject: Request for Income Verification for New York State Medicaid Application


In [20]:
credentials = get_gmail_credentials(
    token_file="token.json",
    scopes=["https://mail.google.com/"],
    client_secrets_file="./data/credentials.json",
)
api_resource = build_resource_service(credentials=credentials)
toolkit = GmailToolkit(api_resource=api_resource)

In [21]:
tools = toolkit.get_tools()

In [22]:
input = {'to':recipient, 'subject':subject,'message':message}

In [23]:
tools[1].invoke(input=input)

'Message sent. Message Id: 18bcf3361e26919d'

In [24]:
user_id = "5nJrkPWa3D0yCKA853mD"